In [1]:
#требует запуска только один раз
pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [11]:

import math
import random

import pygame as pg

FPS = 30

RED = 0xFF0000
BLUE = 0x0000FF
YELLOW = 0xFFC91F
GREEN = 0x00FF00
MAGENTA = 0xFF03B8
CYAN = 0x00FFCC
BLACK = (0, 0, 0)
WHITE = 0xFFFFFF
GREY = 0x7D7D7D
GAME_COLORS = [RED, BLUE, YELLOW, GREEN, MAGENTA, CYAN]

WIDTH = 1366
HEIGHT = 700
SCREENRECT = pg.Rect(0, 0, WIDTH, HEIGHT)

def load_image(file, colorkey=None, scale=1):
    """Загрузка и подготовка изображений load_image(file, colorkey=None, scale=1)
        file - имя файла,  
        colorkey - цвет прозрачности или указать -1, тогда цвет прозрачнои будет = пикселю в левом верхнем углу
        scale - это коэфф маштабирования
    """
    try:
        image = pg.image.load(file)
        image = image.convert()
        
        # изменение размера изображения
        size = image.get_size()
        size = (size[0] * scale, size[1] * scale)
        image = pg.transform.scale(image, size)

        # Установка цвета прозрачности
        if colorkey is not None:
            if colorkey == -1:
                colorkey = image.get_at((0, 0))
            image.set_colorkey(colorkey, pg.RLEACCEL)        
    except pg.error:
        raise SystemExit(f'Не удалось загрузить изображение "{file}" {pg.get_error()}')
    return image

def newmiss():
    m = Missle()
    all.add(m)
    miss.add(m)
def newpriz():
    p=Priz()
    all.add(p)
    priz.add(p)
    
class Missle(pg.sprite.Sprite):
    def __init__(self):
        pg.sprite.Sprite.__init__(self)
        self.image = self.images[0]
        self.image = pg.transform.flip(self.image, 1,0)
        self.image.set_colorkey(WHITE)
        self.image = self.image.copy()
        self.rect = self.image.get_rect()
        # pygame.draw.circle(self.image, RED, self.rect.center, self.radius)
        self.rect.x = WIDTH-10
        self.rect.y = random.randrange(HEIGHT- self.rect.height)
        self.vx = random.randrange(-8, 10)
        self.vy = random.randrange(1, 8)
        self.last_update = pg.time.get_ticks()

    def update(self):

        self.rect.x += self.vx
        self.rect.y += self.vy-1
        if self.rect.top > HEIGHT + 10 or self.rect.left < -25 or self.rect.right > WIDTH + 50:
            self.rect.x = WIDTH-10
            self.rect.y = random.randrange(HEIGHT- self.rect.height)
            self.vx = random.randrange(-8, 10)
            self.vy = random.randrange(1, 8)
            
            
class Priz(pg.sprite.Sprite):
        def __init__(self):
            pg.sprite.Sprite.__init__(self)
            self.type = random.choice(['shield', 'life'])
            self.image = powerup_images[self.type]
            self.rect = self.image.get_rect()
            self.rect.x = random.randrange(WIDTH-self.rect.width)
            self.rect.y = -10
            self.vx = random.randrange(-5, 5)
            self.vy = random.randrange(2, 4)
            self.wait= 10000
            self.live = 10000
            self.last_update = pg.time.get_ticks()
            self.last_update2 = pg.time.get_ticks()
        def update(self):
            self.rect.x+=self.vx
            self.rect.y+=self.vy
            now = pg.time.get_ticks()
            if now-self.last_update> self.wait:
                self.type = random.choice(['shield', 'life'])
                self.image = powerup_images[self.type]
                self.rect = self.image.get_rect()
                self.rect.x = random.randrange(WIDTH-self.rect.width)
                self.rect.y = -10
                self.vx = random.randrange(-5, 5)
                self.vy = random.randrange(2, 4)
                self.last_update = pg.time.get_ticks()
            if self.rect.bottom > HEIGHT or self.rect.top< -10 :
                self.vy *=-1
            if self.rect.top > self.rect.left < 0 or self.rect.right > WIDTH :
                self.vx *=-1
            if now-self.last_update> self.live :
                self.rect.x = WIDTH+50
                self.rect.y = HEIGHT+50
                self.last_update2 = pg.time.get_ticks()
            
class Explosion(pg.sprite.Sprite):
    def __init__(self, center, size):
        pg.sprite.Sprite.__init__(self)
        self.size = size
        self.image = explosion_anim[self.size][0]
        self.rect = self.image.get_rect()
        self.rect.center = center
        self.frame = 0
        self.last_update = pg.time.get_ticks()
        self.frame_rate = 50

    def update(self):
        now = pg.time.get_ticks()
        if now - self.last_update > self.frame_rate:
            self.last_update = now
            self.frame += 1
            if self.frame == len(explosion_anim[self.size]):
                self.kill()
            else:
                center = self.rect.center
                self.image = explosion_anim[self.size][self.frame]
                self.rect = self.image.get_rect()
                self.rect.center = center



class Gun:
    def __init__(self, screen):
        self.screen = screen
        self.f2_power = 10
        self.f2_on = 0
        self.an = 1
        self.color = GREY

    def fire2_start(self, event):
        self.f2_on = 1

    def fire2_end(self, event):
        """Выстрел мячом.

        Происходит при отпускании кнопки мыши.
        Начальные значения компонент скорости мяча vx и vy зависят от положения мыши.
        """
        global balls, bullet
        bullet += 1
        new_ball = Ball(self.screen)
        new_ball.r += 5
        self.an = math.atan2((event.pos[1]-new_ball.y), (event.pos[0]-new_ball.x))
        new_ball.vx = self.f2_power * math.cos(self.an)
        new_ball.vy = - self.f2_power * math.sin(self.an)
        balls.append(new_ball)
        self.f2_on = 0
        self.f2_power = 10

    def targetting(self, event):
        """Прицеливание. Зависит от положения мыши."""
        if event:
            self.an = math.atan((event.pos[1]-450) / (event.pos[0]-20))
        if self.f2_on:
            self.color = RED
        else:
            self.color = GREY

    def draw(self):
        # FIXIT don't know how to do it
        pg.draw.circle(
            self.screen,
            self.color,
            (10, HEIGHT-10),
            10
        )        

    def power_up(self):
        if self.f2_on:
            if self.f2_power < 100:
                self.f2_power += 1
            self.color = RED
        else:
            self.color = GREY


class Target:
    # self.points = 0
    # self.live = 1
    # FIXME: don't work!!! How to call this functions when object is created?
    # self.new_target()

    def new_target(self):
        """ Инициализация новой цели. """
        x = self.x = rnd(600, 780)
        y = self.y = rnd(300, 550)
        r = self.r = rnd(2, 50)
        color = self.color = RED

    def hit(self, points=1):
        """Попадание шарика в цель."""
        self.points += points

    def draw(self):
        ...

        
class Tank(pg.sprite.Sprite):

    speed = 10
    gun_offset_x = 20
    gun_offset_y = 12
    images = []

    def __init__(self):
        pg.sprite.Sprite.__init__(self, self.containers)
        self.image = self.images[0]
        self.rect = self.image.get_rect(midbottom=SCREENRECT.midbottom)
        self.reloading = 0
        self.origtop = self.rect.top
        self.facing = 1
        # добавить дуло танку
        
        self.f2_power = 10
        self.f2_on = 0
        self.an = 1
        self.shield = 100
        self.lives = 3
        self.hidden = False
        self.hide_timer = pg.time.get_ticks()


    def move(self, direction):
        if direction:
            self.facing = direction
        self.rect.move_ip(direction * self.speed, 0)
        self.rect = self.rect.clamp(SCREENRECT)
        if direction < 0:
            self.image = self.images[0]
        elif direction > 0:
            self.image = self.images[1]
    def hide(self):
        # временно скрыть игрока
        self.hidden = True
        self.hide_timer = pg.time.get_ticks()
        self.rect.center = (WIDTH / 2, HEIGHT + 200)

    def gunpos(self):
        x = self.facing * self.gun_offset_x + self.rect.centerx
        y = self.rect.top+self.gun_offset_y
        return x, y
    
    def targetting(self, event):
        pass
        
    def fire2_start(self, event):
        self.f2_on = 1

    def fire2_end(self, event):
        """Выстрел мячом.

        Происходит при отпускании кнопки мыши.
        Начальные значения компонент скорости мяча vx и vy зависят от положения мыши.
        """
        if self.f2_on == 1:
            new_ball = Shot(self.gunpos())
            self.an = math.atan2((event.pos[1]-new_ball.y), (event.pos[0]-new_ball.x))
            new_ball.vx = self.f2_power * math.cos(self.an)
            new_ball.vy = - self.f2_power * math.sin(self.an)
            self.f2_on = 0
            self.f2_power = 5
    
    def power_up(self):
        if self.f2_on:
            if self.f2_power < 100:
                self.f2_power += 1
                
    def update(self):
        # вызывается на каждом цикле игры
        ...

            
class Shot(pg.sprite.Sprite):
    """Снаряды которыми стреляет танк"""

    images = []

    def __init__(self, pos):
        pg.sprite.Sprite.__init__(self, self.containers)
        self.image = self.images[0]
        self.rect = self.image.get_rect(midbottom=pos)
        self.x = pos[0]
        self.y = pos[1]
        self.vx = 1
        self.vy = 1
        self.live = 30

    def update(self):
        """Переместить мяч по прошествии единицы времени.

        Метод описывает перемещение мяча за один кадр перерисовки. То есть, обновляет значения
        self.x и self.y с учетом скоростей self.vx и self.vy, силы гравитации, действующей на мяч,
        и стен по краям окна.
        """
        self.rect.move_ip(self.vx, -self.vy)
        self.vy = self.vy - 1
        if not SCREENRECT.contains(self.rect):
            self.kill()            


# Initialize pygame
if pg.get_sdl_version()[0] == 2:
    pg.mixer.pre_init(44100, 32, 2, 1024)
pg.init()
screen = pg.display.set_mode((WIDTH, HEIGHT))
bullet = 0
balls = []

# доДЕЛАТЬ: загрузку фона
# create the background, tile the bgd image
bgdtile = load_image("data/bg1366x768.jpg")
background = pg.Surface(SCREENRECT.size)
for x in range(0, SCREENRECT.width, bgdtile.get_width()):
     background.blit(bgdtile, (x, 0))
screen.blit(background, (0, 0))
# screen.blit(bgdtile, (0, 0))
pg.display.flip()

# доДЕЛАТЬ: звуки

# Загрузка изображений и назначение спрайтов классам
# (до использования классов, после настройки screen)
img = load_image("data/tank1.png", -1, 0.5)
Tank.images = [img, pg.transform.flip(img, 1, 0)]
Shot.images = [load_image("data/ball.png", -1)]
Missle.images = [load_image("data/spr_missile.png",1)]
score_img = pg.image.load('data/score_fon.png').convert_alpha() # фон для счета
explosion_anim = {}
explosion_anim['lg'] = []
explosion_anim['sm'] = []
explosion_anim['player'] = []
for i in range(9):
    img_lg = load_image('data/regularExplosion0{}.png'.format(i), BLACK, 0.5)
    explosion_anim['lg'].append(img_lg)
    img_sm = pg.transform.scale(img_lg, (32, 32))
    explosion_anim['sm'].append(img_sm)
    
    img = load_image('data/sonicExplosion0{}.png'.format(i), BLACK)
    explosion_anim['player'].append(img)
powerup_images = {}
powerup_images['shield'] = load_image('data/shield.png',WHITE)
powerup_images["life"] = load_image("data/tank1.png", -1, 0.3)
# Initialize Game Groups
shots = pg.sprite.Group()
all = pg.sprite.RenderUpdates()
miss = pg.sprite.Group()
priz = pg.sprite.Group()
for i in range(8):
    newmiss()
newpriz()
score = 0
# assign default groups to each sprite class
Tank.containers = all
Shot.containers = shots, all


clock = pg.time.Clock()
# gun = Gun(screen)
tank1 = Tank()
tank1.rect.left = 0
tank2 = Tank()
tank2.rect.right = WIDTH
# target = Target()
finished = False

score= 0 
font = pg.font.Font(None, 25)  #шрифт для счета очков

while not finished:

    keystate = pg.key.get_pressed()
    # clear/erase the last drawn sprites
    all.clear(screen, background)
    # update all the sprites
    all.update()   
    # handle player input
    
    direction = keystate[pg.K_d] - keystate[pg.K_a]
    tank1.move(direction)
    
    direction = keystate[pg.K_RIGHT] - keystate[pg.K_LEFT]
    tank2.move(direction)
    
    clock.tick(FPS)
    for event in pg.event.get():
        if event.type == pg.QUIT:
            finished = True
        elif event.type == pg.MOUSEBUTTONDOWN:
            but1, but2, but3 = pg.mouse.get_pressed()
            if but1:
                tank1.fire2_start(event)
        elif event.type == pg.MOUSEBUTTONUP:
            tank1.fire2_end(event)
        elif event.type == pg.MOUSEMOTION:
            tank1.targetting(event)
    tank1.power_up()
            # проверьте, не попал ли танк в ракету
    hits = pg.sprite.groupcollide(miss, shots, True, True)
    for hit in hits:
        score += 5
        #random.choice(expl_sounds).play()
        expl = Explosion(hit.rect.center, 'lg')
        all.add(expl)
        newmiss()
    # Проверка на столкновение с призом
    hits = pg.sprite.spritecollide(tank1, priz, True)
    for hit in hits:
        if hit.type == 'shield':
            tank1.shield += random.randrange(30, 50)
            if tank1.shield >= 100:
                tank1.shield = 100
            if hit.type == 'life':
                tank1.life+=1
                

    #  Проверка, не ударил ли моб игрока
    hits = pg.sprite.spritecollide(tank1, miss, True)
    for hit in hits:
        tank1.shield -= 10
        expl = Explosion(hit.rect.center, 'sm')
        all.add(expl)
        newmiss()
        if tank1.shield <= 0:
            death_explosion = Explosion(tank1.rect.center, 'player')
            all.add(death_explosion)
            tank1.hide()
            tank1.lives -= 1
            tank1.shield = 100

    # Если игрок умер, игра окончена
    if tank1.lives == 0 and not death_explosion.alive():
        running = False


    
    # draw the scene
    dirty = all.draw(screen)
    pg.display.update(dirty)   
    
    #отображение очков 1 игрок
    screen.blit(score_img, (0, 0))
    score_text = font.render("Score 1: "+str(score),True,BLACK)
    screen.blit(score_text, (25,20))  
    
    #отображение очков 2 игрок
    screen.blit(score_img, (1220, 0))
    score_text = font.render("Score 2:  0",True,BLACK)
    screen.blit(score_text, (1245,20)) 
    
    pg.display.update()

    
pg.quit()

 